# Syslog CSV Breakdown

This script uses pandas to generate:

- Simple plot of of a `syslog.csv` file. It is assumed that the csv file has been previously created by [syslog-to-csv.py](https://github.com/gm3dmo/syslog-to-csv/blob/main/syslog-to-csv.py)
- A Summary and count of the daemons which wrote to sylog in csv format (`syslog-breakdown.csv`).

You will need to `conda install bokeh` or `pip install bokeh`

In [19]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import pathlib

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

In [ ]:
pd.set_option('display.max_rows', 1000)

cwd = pathlib.Path.cwd()
csv_file = cwd / 'syslog-all.csv'
df = pd.read_csv(csv_file,  dtype={"line_number": int, "line_length": int,  "hostname": "string", "wiped_line": "string" , "daemon": "string"})

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34192365 entries, 0 to 34192364
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   line_number     int64  
 1   line_length     int64  
 2   extracted_date  object 
 3   unix_timestamp  float64
 4   real_date       object 
 5   hostname        string 
 6   daemon          string 
 7   wiped_line      string 
dtypes: float64(1), int64(2), object(2), string(3)
memory usage: 2.0+ GB


Create a pandas datetime column called `real_date` using the `unix_timestamp` column as a source:

In [14]:
df['real_date'] = pd.to_datetime(df['unix_timestamp'],unit='s')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34192365 entries, 0 to 34192364
Data columns (total 8 columns):
 #   Column          Dtype         
---  ------          -----         
 0   line_number     int64         
 1   line_length     int64         
 2   extracted_date  object        
 3   unix_timestamp  float64       
 4   real_date       datetime64[ns]
 5   hostname        string        
 6   daemon          string        
 7   wiped_line      string        
dtypes: datetime64[ns](1), float64(1), int64(2), object(1), string(3)
memory usage: 2.0+ GB


Create the time period *buckets* in which to group the data. In this script We've chosen `600S` (10 minutes) for the granularity of the bucket. Other frequencies can be chosen and are documented in the [offset-aliases](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases). To choose a different *bucket size*, simply replace the `600S` with a new value where you see `freq='600S'`

In [15]:
# Set the sample frequency to 10 minutes (600 seconds)
sample_frequency = '300S'
buckets = df.groupby([pd.Grouper(key='real_date', axis=0, freq=sample_frequency),'daemon']).count()

#### Which daemons are producing the most messages per 10 minutes?

In [16]:
buckets_of_wiped_line = df.groupby([pd.Grouper(key='real_date', axis=0, freq=sample_frequency),'daemon'])['wiped_line'].count().unstack()

### Generate summaries of the bucket data

In [18]:
daemon_plot = buckets_of_wiped_line.hvplot.scatter(x = 'real_date', by='daemon', line_width=2, title="daemon lines written to syslog", width=1600, height=1200)
daemon_plot

:NdOverlay   [daemon]
   :Scatter   [real_date]   (value)

In [ ]:
# Create a csv file of the buckets
buckets_of_wiped_line.to_csv('syslog-breakdown.csv')